# CSE 158, Fall 2019: Homework 4

### **Name:** Hongjian Cui &ensp;&ensp;&ensp;&ensp; **PID:** U08398995 

In [1]:
import json
import numpy as np
import random
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model
from collections import defaultdict

In [2]:
def parseData(fname):
    for l in open(fname):
        yield eval(l)

In [3]:
path = "/home/cui/Projects/PycharmProjects/CSE-158/data/train_Category.json"

In [4]:
dataset = list(parseData(path))

In [5]:
data = dataset[:10000]

In [6]:
punctuation = set(string.punctuation)
corpus = []

for d in data:
    text = d['review_text']
    text = text.lower()
    text = [c for c in text if not (c in punctuation)]
    text = ''.join(text)
    corpus.append(text)

In [7]:
corpus[1]

'pretty decent the ending seemed a little rush but a good ending to the first trilogy in this series the fact that most of the time it is a military fantasy makes it interesting also all of the descriptions of food just make me hungry'

## Question 1:

In [8]:
vec = CountVectorizer(ngram_range=(2, 2), dtype=np.uint16)

In [9]:
X = vec.fit_transform(corpus)

In [10]:
bigram_dic = vec.vocabulary_

In [11]:
bigram_dic

{'genuinely enthralling': 179383,
 'enthralling if': 143297,
 'if collins': 218676,
 'collins or': 93268,
 'or bernard': 333461,
 'bernard did': 58842,
 'did invent': 121687,
 'invent this': 232352,
 'this out': 465827,
 'out of': 337953,
 'of whole': 325661,
 'whole cloth': 515212,
 'cloth they': 91985,
 'they deserve': 462487,
 'deserve medal': 118119,
 'medal for': 285952,
 'for imagination': 167965,
 'imagination lets': 220783,
 'lets leave': 262384,
 'leave the': 260265,
 'the veracity': 457751,
 'veracity aside': 496235,
 'aside for': 41451,
 'for moment': 168320,
 'moment always': 294007,
 'always touchy': 18288,
 'touchy subject': 479229,
 'subject when': 432303,
 'when it': 511796,
 'it comes': 238293,
 'comes to': 94088,
 'to real': 474940,
 'real life': 373902,
 'life stories': 264221,
 'stories of': 428129,
 'of the': 325145,
 'the occult': 454522,
 'occult and': 319127,
 'and talk': 29109,
 'talk about': 439594,
 'about the': 4527,
 'the contents': 450359,
 'contents the':

In [12]:
len(bigram_dic)

533839

In [13]:
wordCount = sorted(bigram_dic.items(),key = lambda x:x[1], reverse = True)

In [14]:
wordCount[:5]

[('zzzaps continuing', 533838),
 ('zywych lub', 533837),
 ('zyndu is', 533836),
 ('zyndu are', 533835),
 ('zydy hwslh', 533834)]

## Question 2:

In [15]:
vec = CountVectorizer(ngram_range=(2, 2), dtype=np.uint16, 
                      max_features=1000)
X = vec.fit_transform(corpus)
X = X.toarray()
y = [d['rating'] for d in data]

In [16]:
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
predictions = clf.predict(X)

In [17]:
def MSE(predictions, targets):
    return ((predictions - targets) ** 2).mean()

In [18]:
print ("The MSE of the training set is {:.3f}."
                       .format(MSE(predictions, y)))

The MSE of the training set is 6.283.


## Question 3:

In [19]:
vec = CountVectorizer(ngram_range=(1, 2), max_features=1000)
X = vec.fit_transform(corpus)
X = X.toarray()
y = [d['rating'] for d in data]

In [20]:
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
predictions = clf.predict(X)

In [21]:
print ("The MSE of the training set is {:.3f}."
                       .format(MSE(predictions, y)))

The MSE of the training set is 5.364.


## Question 4:

In [22]:
tfidf_vec = TfidfVectorizer(ngram_range=(1, 1), dtype=np.float32)
X_tfidf = tfidf_vec.fit_transform(corpus)

In [23]:
idf = tfidf_vec.idf_ - 1

In [24]:
dict_idf = dict(zip(tfidf_vec.get_feature_names(), idf))

In [25]:
dict_idf['stories']

2.5718725

In [26]:
dict_idf['magician']

6.074946

In [27]:
dict_idf['psychic']

5.952344

In [28]:
dict_idf['writing']

2.296703

In [29]:
dict_idf['wonder']

4.062946

In [30]:
X_tfidf = X_tfidf.toarray()

In [31]:
dict_tfidf_first = dict(zip(tfidf_vec.get_feature_names(), X_tfidf[0]))

In [32]:
dict_tfidf_first['stories']

0.04845343

In [33]:
dict_tfidf_first['magician']

0.09597358

In [34]:
dict_tfidf_first['psychic']

0.18862091

In [35]:
dict_tfidf_first['writing']

0.044720683

In [36]:
dict_tfidf_first['wonder']

0.06868025

## Question 5:

In [37]:
tfidf_vec = TfidfVectorizer(ngram_range=(1, 1), dtype=np.float32, 
                            max_features=1000)
X_tfidf = tfidf_vec.fit_transform(corpus)
X_tfidf = X_tfidf.toarray()
y = [d['rating'] for d in data]

In [38]:
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X_tfidf, y)
predictions = clf.predict(X_tfidf)

In [39]:
print ("The MSE of the training set is {:.3f}."
                       .format(MSE(predictions, y)))

The MSE of the training set is 1.523.


## Question 6:

In [40]:
def cosineSimilarity(s1, s2):
    numer = sum(s1 * s2)
    denom = np.sqrt(sum(s1 ** 2)) * np.sqrt(sum(s2 ** 2))
    
    if denom == 0:
        return 0
    else:
        return numer / denom

In [41]:
similarity = [cosineSimilarity(X_tfidf[0], d) for d in X_tfidf]

In [42]:
similarity[0] = 0

In [43]:
maxSimilarity = max(similarity)

In [44]:
maxSimilarity

0.5295288962174183

In [45]:
index = similarity.index(maxSimilarity)

In [46]:
data[index]['review_id']

'r64325341'